## Callbacksモジュールを使って外部ライブラリーと連携

In [ ]:
import chainlit as cl
from langchain.agents import AgentType
from langchain.agents import initialize_agent
from langchain.agents import load_tools
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    temperature=0,
    #model="gpt-3.5-turbo"
    model="gpt-4o"
)

tools=load_tools(
    [
        "serpapi",
    ]
)
#
#Agent初期化
agent = initialize_agent(
    tools=tools,
    llm=chat,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="Agentの初期化が終了しました").send()

@cl.on_message
async def on_message(input_message):
    result=agent.run(
        input_message,
        callbacks=[
            cl.LangchainCallbackHandler()
        ]
    )
    await cl.Message(content=result).send()

## ログをターミナルに表示

In [8]:
#python fileName で実行
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI

class LogCallbackHandler(BaseCallbackHandler):
    #
    def on_chat_model_start(self, serialized, messages, **kwargs):
        print("ChatModelの実行を開始します......")
        print(f"入力:{messages}")
    #
    def on_chain_start(self, serialized, inputs, **kwargs):
        print("Chainの実行を開始します......")
        print(f"入力:{inputs}")

#コールバックにコールバッククラス
chat = ChatOpenAI(
    model="gpt-4o",
    callbacks=[
        LogCallbackHandler()
    ]
)

#ユーザーメッセージ
result = chat(
    [
        HumanMessage(content="こんにちは！"),
    ]
)

print(result.content)

ChatModelの実行を開始します......
入力:[[HumanMessage(content='こんにちは！')]]
こんにちは！今日はどんなお手伝いができますか？
